In [16]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

# Auth
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('../gspread_creds.json', scope)
client = gspread.authorize(creds)

In [ ]:
hb_gmbh = pd.read_excel('../../Moodys_name_to_handbuch/output/manual/gmbh_validated.xlsx', sheet_name='validated')
hb_gmbh = hb_gmbh[~hb_gmbh['validated'].isin(['Z', 'ZN'])]

hb_gmbh["is_duplicate"] = hb_gmbh.duplicated(subset=['Master German firm name', 'Master US firm name'])
# dropping duplicates
hb_gmbh = hb_gmbh[hb_gmbh["is_duplicate"] == False]

In [18]:
hb_gmbh_full = pd.read_csv('../../Handbuch_GmbH_1932/output/firms_structured.csv')

In [19]:
hb_gmbh_merged = hb_gmbh.merge(hb_gmbh_full, left_on='german_name', right_on='firm_name', how='left')
hb_gmbh_merged = hb_gmbh_merged.rename(columns={'validated': 'link_type'}).assign(org_type='GmbH')

In [20]:
hb_gmbh_merged

,german_name,Master German firm name,US_name,Master US firm name,openai_unclean,openai_clean,sentence_transformers_unclean,sentence_transformers_clean,assessment,validated_old,...,page,column,firm_name,location,date,capital,business,persons,type,org_type
0,Agfa Photo Gesellschaft,Agfa Photo Gesellschaft,Agfa-Ansco Corporation,Agfa-Ansco Corporation,False,True,False,False,POSSIBLE,1.0,...,25,right,Agfa Photo Gesellschaft,"Breslau, Wallstr. 1",23.12.25,RM 5000,Vertrieb von Photoartikeln der I. G. Farbenind...,Curt Voigt; Emil Steffen,1 Gf.,GmbH
1,Agfa Photo Gesellschaft,Agfa Photo Gesellschaft,Agfa-Ansco Corporation,Agfa-Ansco Corporation,False,True,False,False,POSSIBLE,1.0,...,25,right,Agfa Photo Gesellschaft,"Dresden, Christianstr. 27",23.12.25,RM 5000,Vertrieb von Photoartikeln der I. G. Farbenind...,M. Gustav Vogel,1 Gf.,GmbH
2,Agfa Photo Gesellschaft,Agfa Photo Gesellschaft,Agfa-Ansco Corporation,Agfa-Ansco Corporation,False,True,False,False,POSSIBLE,1.0,...,25,right,Agfa Photo Gesellschaft,"Düsseldorf, Haroldstr. 17",8.6.25,RM 5000,Vertrieb von Photoartikeln der A. G. für Anili...,Hans Grieb; Richard Häusgen,1 Gf.,GmbH
3,Agfa Photo Gesellschaft,Agfa Photo Gesellschaft,Agfa-Ansco Corporation,Agfa-Ansco Corporation,False,True,False,False,POSSIBLE,1.0,...,25,right,Agfa Photo Gesellschaft,"Frankfurt a. M., Seilerstr. 26",23.12.25,RM 5000,Vertrieb von Photoartikeln der I. G. Farbenind...,Wilhelm Stanigel,1 Gf.,GmbH
4,Agfa Photo Gesellschaft,Agfa Photo Gesellschaft,Agfa-Ansco Corporation,Agfa-Ansco Corporation,False,True,False,False,POSSIBLE,1.0,...,25,right,Agfa Photo Gesellschaft,"Hamburg, Grindelallee 43",23.12.25,RM 5000,Vertrieb von Photoartikeln der I. G. Farbenind...,G. Philipp Knoch; R. A. F. Robert Rödiger,1 Gf.,GmbH
5,Agfa Photo Gesellschaft,Agfa Photo Gesellschaft,Agfa-Ansco Corporation,Agfa-Ansco Corporation,False,True,False,False,POSSIBLE,1.0,...,25,right,Agfa Photo Gesellschaft,"Hannover, Lavesstr. 54",23.12.25,RM 5000,Vertrieb von Photoartikeln der I. G. Farbenind...,Walter Barrenscheen,1 Gf.,GmbH
6,Agfa Photo Gesellschaft,Agfa Photo Gesellschaft,Agfa-Ansco Corporation,Agfa-Ansco Corporation,False,True,False,False,POSSIBLE,1.0,...,25,right,Agfa Photo Gesellschaft,"Köln, Hohenstaufenring 66",23.12.25,RM 5000,Vertrieb von Photoartikeln der I. G. Farbenind...,Kurt Wisniewski,1 Gf.,GmbH
7,Agfa Photo Gesellschaft,Agfa Photo Gesellschaft,Agfa-Ansco Corporation,Agfa-Ansco Corporation,False,True,False,False,POSSIBLE,1.0,...,25,right,Agfa Photo Gesellschaft,Königsberg i. Pr.,23.12.25,RM 5000,Vertrieb von Photoartikeln der I. G. Farbenind...,Carl Drognitz,1 Gf.,GmbH
8,Agfa Photo Gesellschaft,Agfa Photo Gesellschaft,Agfa-Ansco Corporation,Agfa-Ansco Corporation,False,True,False,False,POSSIBLE,1.0,...,25,right,Agfa Photo Gesellschaft,"Leipzig, Kreuzstr. 2",23.12.25,RM 5000,Vertrieb von Photoartikeln der I. G. Farbenind...,Otto Oertel; Kurt Habersang,1 Gf.,GmbH
9,Agfa Photo Gesellschaft,Agfa Photo Gesellschaft,Agfa-Ansco Corporation,Agfa-Ansco Corporation,False,True,False,False,POSSIBLE,1.0,...,25,right,Agfa Photo Gesellschaft,"München, Barerstr. 15",16.9.25,RM 5000,Vertrieb von Photoartikeln der A. G. für Anili...,Heinr. Wiedenmann,1 Gf.,GmbH


In [21]:
hb_gmbh_merged = hb_gmbh_merged.rename(columns={'Master US firm name': 'US Company', 
                                                'Master German firm name': 'German subsidiary',
                                                'german_name': 'Affiliated German firm (as in book)',
                                                'US_name': 'US firm (as in book)'})

hb_gmbh_merged = hb_gmbh_merged[['German subsidiary', 'US Company', 'link_type', 'org_type', 'US firm (as in book)', 'Affiliated German firm (as in book)', 'other sources', 'notes', 'page', 'location', 'date', 'capital', 'business', 'persons', 'type']]

In [22]:
hb_gmbh_merged.to_csv('../output/hb_gmbh_cleaned.csv', index=False)